In [6]:
library(ggplot2)


intensity = 2^(-8:4)

flopsmax = 16 # GB/s
        
memband <- data.frame(intensity, intensity * 16 )
colnames(memband) <- c("intesnity","FLOPS")

aplot <- ggplot(memband, aes(x=intensity, y=FLOPS, color="off-chip bw limited")) 
         + geom_line() + coord_trans(x="log2", y="log2") + ylim(0,20)

print(aplot)



ERROR: Error: Cannot use `+.gg()` with a single argument. Did you accidentally put + on a new line?
